In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

color_pal = sns.color_palette()

In [2]:
%%time
ward = pd.read_csv("Original Data/nl_sum_city_wards.csv", low_memory=False)
tehsils = pd.read_csv("Original Data/nl_sum_tehsil_wo_cities.csv", low_memory=False)
pincodes = pd.read_csv("Original Data/nl_sum_pincodes.csv", low_memory=False)

Wall time: 4.53 s


# Data Preprocessing

## Data Cleaning

In [3]:
def base_preprocessing(df):
    df2 = df.copy()
    df2 = df2.drop(['.geo', 'system:index'], axis = 1)
    df2 = df2.drop(df.filter(like='cf_cvg').columns, axis=1)
    return df2

In [4]:
wards = base_preprocessing(ward)
wards = wards.drop(['ward_no', 'status'], axis = 1)

# Get the value from the 'Area' column
area_value = wards['Area']

# Multiply columns with 'avg_rad' in their headers by 'Area' and multiply by 10^-7
wards.loc[:, wards.columns.str.contains('avg_rad')] = wards.loc[:, wards.columns.str.contains('avg_rad')].multiply(area_value, axis=0) * 1e-7

In [5]:
columns = ['id', 'ward_name', 'city', 'state', 'Area']
columns_wo_id = ['ward_name', 'city', 'state', 'Area']
# Divide the dataframe by these two columns, Kepping 'id' in both
wards_df_info = wards[columns].copy()
wards_raw_data = wards.copy()
wards_raw_data = wards_raw_data.drop(columns = columns_wo_id)

In [6]:
tehsil = base_preprocessing(tehsils)
tehsil = tehsil.drop(["district_c", "state_code"], axis = 1)

# Get the value from the 'Area' column
area_value = tehsil['teh_area']

# Multiply columns with 'avg_rad' in their headers by 'Area' and multiply by 10^-7
tehsil.loc[:, tehsil.columns.str.contains('avg_rad')] = tehsil.loc[:, tehsil.columns.str.contains('avg_rad')].multiply(area_value, axis=0) * 1e-7

In [7]:
tehsil_id = tehsil['id'].to_list()
columns = ['id', 'tehsil_nam', 'state_name', 'tehsil_cod', 'teh_area', "district_n"]
columns_wo_id = ['tehsil_nam', 'state_name', 'tehsil_cod', 'teh_area', "district_n"]
# Divide the dataframe by these two columns, Kepping 'id' in both
tehsil_df_info = tehsil[columns].copy()
tehsil_raw_data = tehsil.copy()
tehsil_raw_data = tehsil_raw_data.drop(columns=columns_wo_id)

In [8]:
pincode = base_preprocessing(pincodes)
pincode = pincode.drop(['state_name', 'state_code', 'pin_type', 'district_c', 'class', 'area2'], axis = 1)

# Get the value from the 'Area' column
area_value = pincode['pin_area']

# Multiply columns with 'avg_rad' in their headers by 'Area' and multiply by 10^-7
pincode.loc[:, pincode.columns.str.contains('avg_rad')] = pincode.loc[:, pincode.columns.str.contains('avg_rad')].multiply(area_value, axis=0) * 1e-7

In [9]:
columns = ['id', 'pincode', 'state_abb', 'district_n', 'pin_name', 'pin_area']
columns_wo_id = ['state_abb', 'pincode', 'district_n', 'pin_name', 'pin_area']
# Divide the dataframe by these two columns, Kepping 'id' in both
pin_df_info = pincode[columns]
pin_raw_data = pincode.copy()
pin_raw_data = pin_raw_data.drop(columns= columns_wo_id)

## Converting the Dataframes
> The current format is not the best for visualization or calculations. Hence, we transpose the dataframes, and convert it into a datettime type with value columns

In [10]:
# All the dates in YYYYMMDD format
date_list = [20120401, 20120501, 20120601, 20120701, 20120801, 20120901, 20121001, 20121101, 20121201,
    20130101, 20130201, 20130301, 20130401, 20130501, 20130601, 20130701, 20130801, 20130901, 20131001, 20131101, 20131201,
    20140101, 20140201, 20140301, 20140401, 20140501, 20140601, 20140701, 20140801, 20140901, 20141001, 20141101, 20141201, 
    20150101, 20150201, 20150301, 20150401, 20150501, 20150601, 20150701, 20150801, 20150901, 20151001, 20151101, 20151201, 
    20160101, 20160201, 20160301, 20160401, 20160501, 20160601, 20160701, 20160801, 20160901, 20161001, 20161101, 20161201, 
    20170101, 20170201, 20170301, 20170401, 20170501, 20170601, 20170701, 20170801, 20170901, 20171001, 20171101, 20171201, 
    20180101, 20180201, 20180301, 20180401, 20180501, 20180601, 20180701, 20180801, 20180901, 20181001, 20181101, 20181201, 
    20190101, 20190201, 20190301, 20190401, 20190501, 20190601, 20190701, 20190801, 20190901, 20191001, 20191101, 20191201, 
    20200101, 20200201, 20200301, 20200401, 20200501, 20200601, 20200701, 20200801, 20200901, 20201001, 20201101, 20201201, 
    20210101, 20210201, 20210301, 20210401, 20210501, 20210601, 20210701, 20210801, 20210901, 20211001, 20211101, 20211201, 
    20220101, 20220201, 20220301, 20220401, 20220501, 20220601, 20220701, 20220801, 20220901, 20221001, 20221101, 20221201,
    20230101, 20230201 ]

In [11]:
nl_series_wards = wards_raw_data.T
# Assign the row at 'row_index' as the new column headers
nl_series_wards.columns = nl_series_wards.loc['id']
# Drop the 'row_index' row from the DataFrame
nl_series_wards = nl_series_wards.drop('id')
# Extract the date part from row indexes
date_part = nl_series_wards.index.str.split('_').str[0]
nl_series_wards.index = pd.to_datetime(date_part, format='%Y%m%d')
nl_series_wards.head(7)

id,7930.0,7931.0,7932.0,7933.0,7935.0,7936.0,7940.0,7942.0,7944.0,7945.0,...,3333.0,3340.0,3341.0,3343.0,3347.0,3349.0,3350.0,3351.0,3353.0,3361.0
2012-04-01,228.194633,16.575666,225.891985,97.099543,1122.238605,20.080662,54.239890,31.737696,5758.510333,29.336071,...,0.761303,1.298041,0.187498,2.462176,1.431461,0.510079,0.615521,2.600839,3.012453,3.494639
2012-05-01,152.847481,12.783665,173.764587,75.299474,840.046076,15.296669,38.330599,22.128599,4183.420194,21.303442,...,0.558004,1.182626,0.157932,2.004145,1.178309,0.416352,0.345541,1.962977,2.340842,3.354116
2012-06-01,162.459724,12.591488,155.652966,77.381551,757.938407,15.120473,35.826675,21.138964,4275.138770,19.476142,...,0.444162,0.535917,0.086953,2.044687,0.765072,0.261983,0.266280,1.824316,1.519913,2.509684
2012-07-01,196.447394,11.761018,100.039266,92.338868,731.468523,16.190184,37.590942,26.502692,1776.096239,18.338348,...,0.565309,0.734717,0.125068,1.966239,1.122202,0.337192,0.310109,1.832933,2.180074,2.196870
2012-08-01,62.475985,3.649421,74.889849,23.770446,96.124036,7.660006,10.049300,16.172426,2187.881535,12.284047,...,0.781058,0.924807,0.207623,1.881168,1.081659,0.476346,0.208022,1.647995,1.787867,2.728866
2012-09-01,211.177584,14.092453,194.850206,87.104858,987.100766,18.355643,42.703527,25.016326,5010.794265,22.334403,...,0.819086,1.023178,0.161674,2.314964,1.261311,0.394484,0.435836,2.730743,2.294217,2.924751
2012-10-01,204.885359,15.672524,208.354679,93.231636,1200.255305,19.386067,49.093314,30.008586,5712.222309,27.946348,...,0.487484,0.790445,0.115057,1.782550,0.871831,0.217938,0.368039,1.669238,1.363640,2.987833


In [12]:
nl_series_tehsil = tehsil_raw_data.T
nl_series_tehsil.set_axis(tehsil_id, axis = "columns")
# Assign the row at 'row_index' as the new column headers
nl_series_tehsil.columns = nl_series_tehsil.loc['id']
# Drop the 'row_index' row from the DataFrame
nl_series_tehsil = nl_series_tehsil.drop('id')
# Extract the date part from row indexes
date_part = nl_series_tehsil.index.str.split('_').str[0]
nl_series_tehsil.index = pd.to_datetime(date_part, format='%Y%m%d')
nl_series_tehsil.head(7)

id,4323.0,3643.0,3642.0,3644.0,3646.0,3640.0,3641.0,3645.0,3647.0,3648.0,...,128.0,126.0,121.0,127.0,2726.0,120.0,2727.0,2724.0,2725.0,2728.0
2012-04-01,885.082803,0.005871,0.108967,0.427446,0.031688,0.016880,0.005072,0.014909,0.000695,0.004613,...,0.005071,0.008514,0.024130,0.002735,0.032079,0.147857,0.129254,0.180021,0.096843,0.041503
2012-05-01,870.959947,0.005405,0.115265,0.419677,0.031942,0.016616,0.006264,0.015705,0.001022,0.004766,...,0.004504,0.006581,0.048317,0.001328,0.000367,0.054030,0.030501,0.048338,0.022713,0.009487
2012-06-01,181.840737,0.000000,0.000765,0.000000,0.039474,0.024576,0.008776,0.021064,0.001055,0.006737,...,0.003790,0.010582,0.039358,0.002695,0.017727,0.123229,0.081001,0.081670,0.066142,0.037900
2012-07-01,847.611375,0.006443,0.137226,0.533887,0.030131,0.018621,0.007580,0.020938,0.001217,0.005660,...,0.000416,0.004565,-0.023678,0.001726,-0.003405,-0.010743,-0.006942,0.022937,0.040180,0.008273
2012-08-01,797.546340,0.007008,0.156142,0.625125,0.032341,0.022305,0.007792,0.021348,0.001233,0.007066,...,0.001138,0.002792,-0.015636,0.002428,0.000534,0.028547,0.003488,0.066872,0.002994,0.001094
2012-09-01,1192.207801,0.008311,0.182160,0.693382,0.034193,0.023538,0.009853,0.026688,0.001759,0.007556,...,0.004708,0.000391,0.025804,0.000710,0.000745,0.027074,0.015193,0.001249,0.000328,0.000036
2012-10-01,1817.325413,0.010687,0.212510,0.761586,0.032994,0.023579,0.013329,0.032035,0.002481,0.007894,...,0.000051,0.003826,0.009435,0.001103,0.010801,0.050399,0.045933,0.061688,0.071423,0.013501


In [13]:
nl_series_pincode = pin_raw_data.T
# Assign the row at 'row_index' as the new column headers
nl_series_pincode.columns = nl_series_pincode.loc['id']
# Drop the 'row_index' row from the DataFrame
nl_series_pincode = nl_series_pincode.drop('id')
# Extract the date part from row indexes
date_part = nl_series_pincode.index.str.split('_').str[0]
nl_series_pincode.index = pd.to_datetime(date_part, format='%Y%m%d')
nl_series_pincode.head(7)

id,3265.0,3264.0,4582.0,4585.0,4583.0,4569.0,4586.0,4587.0,4615.0,18708.0,...,3164.0,3165.0,3168.0,3170.0,3171.0,3166.0,3169.0,3159.0,18972.0,3177.0
2012-04-01,92.121098,1024.870697,0.041944,0.152026,0.001341,0.025124,0.005317,0.058083,0.004637,0.002293,...,0.003376,0.000579,1.909426e-04,0.000005,0.000010,0.029423,0.010204,0.043783,0.065354,0.006414
2012-05-01,109.036545,1009.207572,0.041192,0.136454,0.001297,0.027748,0.006445,0.050065,0.004459,0.002555,...,0.000798,0.000233,3.849154e-05,0.000004,0.000003,0.005976,0.005858,0.009936,0.016567,0.000656
2012-06-01,9.074227,210.110650,0.000383,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001985,0.000578,1.876859e-04,0.000005,0.000008,0.005271,0.005570,0.039935,0.035947,0.000939
2012-07-01,132.805649,983.586173,0.050873,0.168447,0.001585,0.031956,0.010000,0.069128,0.005232,0.002910,...,0.000980,0.000261,5.990493e-05,0.000005,0.000007,-0.003053,0.003224,0.008475,0.019003,0.000276
2012-08-01,99.077823,925.354700,0.058461,0.208574,0.001683,0.035002,0.010890,0.080053,0.006512,0.003327,...,0.000880,0.000158,1.797793e-06,0.000000,0.000000,0.005002,0.001139,0.001245,0.021654,0.000096
2012-09-01,146.548493,1386.664936,0.065022,0.226644,0.001904,0.045391,0.012943,0.088727,0.006388,0.003741,...,0.000004,0.000000,7.394356e-07,0.000000,0.000000,0.001961,0.000056,0.000102,0.000424,0.000174
2012-10-01,243.380301,2121.959434,0.076109,0.296629,0.001772,0.057630,0.012845,0.106572,0.006680,0.003247,...,0.001452,0.000401,1.820891e-04,0.000006,0.000011,0.006619,0.004725,0.014112,0.034206,0.001265


## Removing erroneous zero values
> Zero values are a result of excessive cloud cover over a particular region in a month, hence making the satellite images darker, causing zero value appearances. This can be offset by taking the average of the values that appear before and after the zero value, as the month on month increase should not be too excessive

In [14]:
def remove_zeros(df):
    # Replace all values equal to 0 with NaN
    df.replace(0, np.nan, inplace=True)
    # Forward fill NaN values
    ffilled = df.ffill()
    # Backward fill NaN values
    bfilled = df.bfill()
    # Calculate the mean of ffill and bfill values
    mean_values = (ffilled + bfilled) / 2
    # Replace NaN values with the mean values
    df.fillna(mean_values, inplace=True)

In [15]:
remove_zeros(nl_series_tehsil)
remove_zeros(nl_series_wards)
remove_zeros(nl_series_pincode)

## Calculating Yearly Averages

In [16]:
# Calculate the yearly average for all wards
ward_yearly_average = nl_series_wards.groupby(pd.Grouper(freq='Y')).mean()
# Calculate the yearly average for all tehsils
tehsil_yearly_average = nl_series_tehsil.groupby(pd.Grouper(freq='Y')).mean()
# Calculate the yearly average for all pincodes
pincode_yearly_average = nl_series_pincode.groupby(pd.Grouper(freq='Y')).mean()

# Final Dataset Generation

In [17]:
# Resetiing the dataframes by transposing it again

ward_yearly_average_final = ward_yearly_average.T
ward_monthly_final = nl_series_wards.T

tehsil_monthly_final = nl_series_tehsil.T
tehsil_yearly_average_final = tehsil_yearly_average.T

pincode_monthly_final = nl_series_pincode.T
pincode_yearly_average_final = pincode_yearly_average.T

In [18]:
# Formatting the dataframe for easy reading
yearly_col= ["2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]

monthly_col= ["Apr'12", "May'12", "Jun'12", "Jul'12", "Aug'12", "Sep'12", "Oct'12", "Nov'12", "Dec'12",
             "Jan'13", "Feb'13", "Mar'13", "Apr'13", "May'13", "Jun'13", "Jul'13", "Aug'13", "Sep'13", "Oct'13", "Nov'13", "Dec'13",
             "Jan'14", "Feb'14", "Mar'14", "Apr'14", "May'14", "Jun'14", "Jul'14", "Aug'14", "Sep'14", "Oct'14", "Nov'14", "Dec'14",
             "Jan'15", "Feb'15", "Mar'15", "Apr'15", "May'15", "Jun'15", "Jul'15", "Aug'15", "Sep'15", "Oct'15", "Nov'15", "Dec'15",
             "Jan'16", "Feb'16", "Mar'16", "Apr'16", "May'16", "Jun'16", "Jul'16", "Aug'16", "Sep'16", "Oct'16", "Nov'16", "Dec'16",
             "Jan'17", "Feb'17", "Mar'17", "Apr'17", "May'17", "Jun'17", "Jul'17", "Aug'17", "Sep'17", "Oct'17", "Nov'17", "Dec'17",
             "Jan'18", "Feb'18", "Mar'18", "Apr'18", "May'18", "Jun'18", "Jul'18", "Aug'18", "Sep'18", "Oct'18", "Nov'18", "Dec'18",
             "Jan'19", "Feb'19", "Mar'19", "Apr'19", "May'19", "Jun'19", "Jul'19", "Aug'19", "Sep'19", "Oct'19", "Nov'19", "Dec'19",
             "Jan'20", "Feb'20", "Mar'20", "Apr'20", "May'20", "Jun'20", "Jul'20", "Aug'20", "Sep'20", "Oct'20", "Nov'20", "Dec'20",
             "Jan'21", "Feb'21", "Mar'21", "Apr'21", "May'21", "Jun'21", "Jul'21", "Aug'21", "Sep'21", "Oct'21", "Nov'21", "Dec'21",
             "Jan'22", "Feb'22", "Mar'22", "Apr'22", "May'22", "Jun'22", "Jul'22", "Aug'22", "Sep'22", "Oct'22", "Nov'22", "Dec'22",
             "Jan'23", "Feb'23"]

tehsil_yearly_average_final.columns = yearly_col
tehsil_monthly_final.columns = monthly_col

ward_yearly_average_final.columns = yearly_col
ward_monthly_final.columns = monthly_col

pincode_yearly_average_final.columns = yearly_col
pincode_monthly_final.columns = monthly_col

In [19]:
# Perform the merge based on the index columns
tehsil_final = pd.merge(tehsil_yearly_average_final, tehsil_monthly_final, left_index=True, right_index=True)
ward_final = pd.merge(ward_yearly_average_final, ward_monthly_final, left_index=True, right_index=True)
pincode_final = pd.merge(pincode_yearly_average_final, pincode_monthly_final, left_index=True, right_index=True)

# Data Calculations

## CAGR
> Compounded Annual Growth Rate of nighlight

In [20]:
def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods - 1)) - 1

In [21]:
def create_cagr_df(original_df):
    years = original_df.columns.astype(int)
    years = [2022-i for i in years]
    years = [x for x in years if x>1]
    years = [2022-i for i in years]
    
    cagr_df = pd.DataFrame(index = original_df.index, columns = years)
    
    for i in range(len(years)):
        start_year = years[i]
        end_year = 2022
        period = end_year - start_year
        cagr_values = []
        for _, row in original_df.iterrows():
            cagr_id = cagr(row[str(start_year)], row[str(end_year)], period)
            cagr_values.append(cagr_id)
            
        cagr_df[start_year] = cagr_values

    return cagr_df

In [22]:
%%time
cagr_df_pincode = create_cagr_df(pincode_yearly_average_final)
cagr_df_ward = create_cagr_df(ward_yearly_average_final)
cagr_df_tehsil = create_cagr_df(tehsil_yearly_average_final)

Wall time: 7.31 s


## SAGR
> Simple Annual Growth Rate of nighlight. This is an yearly metric, hence measures the percentage of growth in the given year compared to the previous year

In [23]:
def create_sagr_df(original_df):
    years = original_df.columns.astype(int)
    years = [2022-i for i in years]
    years = [x for x in years if x>=0]
    years = [2022-i for i in years]
    
    sagr_df = pd.DataFrame(index = original_df.index, columns = years[1:])
    
    for i in range(len(years)):
        start_year = years[i-1]
        end_year = years[i]
        sagr_values = []
        for _, row in original_df.iterrows():
            sagr = (row[str(end_year)] - row[str(start_year)])/row[str(start_year)]
            sagr_values.append(sagr)
            
        sagr_df[start_year] = sagr_values

    return sagr_df

In [24]:
%%time
sagr_df_pincode = create_sagr_df(pincode_yearly_average_final)
sagr_df_ward = create_sagr_df(ward_yearly_average_final)
sagr_df_tehsil = create_sagr_df(tehsil_yearly_average_final)

Wall time: 12.7 s


# Complete percentage change
> This metric measures the value of the given year compared to the value in 2022.

In [25]:
def create_sagr22_df(original_df):
    years = original_df.columns.astype(int)
    years = [2022-i for i in years]
    years = [x for x in years if x>0]
    years = [2022-i for i in years]
    
    sagr22_df = pd.DataFrame(index = original_df.index, columns = years[:])
    
    for i in range(len(years)):
        start_year = years[i]
        end_year = 2022
        sagr22_values = []
        for _, row in original_df.iterrows():
            sagr22 = ((row[str(end_year)] - row[str(start_year)])/row[str(start_year)])/(end_year-start_year)
            sagr22_values.append(sagr22)
            
        sagr22_df[start_year] = sagr22_values

    return sagr22_df

In [26]:
%%time
sagr22_df_pincode = create_sagr22_df(pincode_yearly_average_final)
sagr22_df_ward = create_sagr22_df(ward_yearly_average_final)
sagr22_df_tehsil = create_sagr22_df(tehsil_yearly_average_final)

Wall time: 8.39 s


# Saving all the Files

In [28]:
sagr_df_pincode.to_csv('Calculations/SAGR/pincode.csv')
sagr_df_tehsil.to_csv('Calculations/SAGR/tehsil.csv')
sagr_df_ward.to_csv('Calculations/SAGR/ward.csv')

In [30]:
cagr_df_pincode.to_csv('Calculations/CAGR/pincode.csv')
cagr_df_tehsil.to_csv('Calculations/CAGR/tehsil.csv')
cagr_df_ward.to_csv('Calculations/CAGR/ward.csv')

In [32]:
sagr22_df_pincode.to_csv('Calculations/Percent/pincode.csv')
sagr22_df_tehsil.to_csv('Calculations/Percent/tehsil.csv')
sagr22_df_ward.to_csv('Calculations/Percent/ward.csv')

In [34]:
tehsil_final.to_csv('Transformed_Data/Tehsil_data.csv')
ward_final.to_csv('Transformed_Data/Ward_data.csv')
pincode_final.to_csv('Transformed_Data/Pincode_data.csv')

In [37]:
pin_df_info.to_csv('Transformed_Data/Pincode_metadata.csv')
tehsil_df_info.to_csv('Transformed_Data/Tehsil_metadata.csv')
wards_df_info.to_csv('Transformed_Data/Ward_metadata.csv')